<a href="https://colab.research.google.com/github/ramin77786408/Artificial-Intelligence-Deep-Learning-Machine-Learning-Tutorials/blob/master/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("data.csv")
df

,time,open,high,low,close,senkou_span_a,senkou_span_b,kijun-sen,tenkan-sen,senkou_span_a_1,...,senkou_span_a_9,senkou_span_b_9,kijun-sen_9,tenkan-sen_9,senkou_span_a_10,senkou_span_b_10,kijun-sen_10,tenkan-sen_10,buy_profit,sell_profit
0,0.250000,1.21378,1.21404,1.21372,1.21398,1.213215,1.212855,1.212855,1.213575,1.213205,...,1.212920,1.212715,1.212715,1.213125,1.212713,1.212585,1.212585,1.212840,0.0,0.0
1,0.250000,1.21398,1.21400,1.21386,1.21394,1.213215,1.212855,1.212855,1.213575,1.213215,...,1.212935,1.212715,1.212715,1.213155,1.212920,1.212715,1.212715,1.213125,0.0,0.0
2,0.250000,1.21394,1.21399,1.21372,1.21373,1.213233,1.212855,1.212890,1.213575,1.213215,...,1.212970,1.212715,1.212715,1.213225,1.212935,1.212715,1.212715,1.213155,0.0,0.0
3,0.250000,1.21373,1.21376,1.21343,1.21346,1.213233,1.212855,1.212890,1.213575,1.213233,...,1.213015,1.212715,1.212715,1.213315,1.212970,1.212715,1.212715,1.213225,0.0,0.0
4,0.291667,1.21348,1.21375,1.21337,1.21348,1.213255,1.212855,1.212890,1.213620,1.213233,...,1.213027,1.212715,1.212715,1.213340,1.213015,1.212715,1.212715,1.213315,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99959,0.833333,1.04942,1.04948,1.04910,1.04918,1.050017,1.048535,1.050235,1.049800,1.050082,...,1.049440,1.048215,1.048740,1.050140,1.049338,1.048050,1.048535,1.050140,0.0,0.0
99960,0.875000,1.04917,1.04938,1.04903,1.04936,1.049990,1.048535,1.050215,1.049765,1.050017,...,1.049573,1.048275,1.049005,1.050140,1.049440,1.048215,1.048740,1.050140,0.0,0.0
99961,0.875000,1.04935,1.04941,1.04929,1.04935,1.049888,1.048535,1.050145,1.049630,1.049990,...,1.049573,1.048425,1.049005,1.050140,1.049573,1.048275,1.049005,1.050140,0.0,0.0
99962,0.875000,1.04936,1.04940,1.04924,1.04936,1.049853,1.048535,1.050145,1.049560,1.049888,...,1.049753,1.048460,1.049255,1.050250,1.049573,1.048425,1.049005,1.050140,0.0,0.0


# Environment

In [ ]:
from gym import spaces
import numpy as np
import random
import pandas as pd
import time

RED = "\033[31m"
RESET = "\033[0m"

point = 100000
episode_reward = 0.0
running_reward = 0.0


class TrainForex:

    def __init__(self, max_step_per_episode=20, lot=0.01, df=None):
        global episode_reward
        super(TrainForex, self).__init__()
        self.max_step_per_episode = max_step_per_episode
        self.lot = lot*point
        self.tp = 100
        self.action_space = spaces.Discrete(6)
        self.price_action = []
        self.steps = 0
        self.df = df
        self.reward=0.0
        self.done = False
        self.start_point = 0
        self.market = 0.0
        self.states = np.zeros(df.shape[1], dtype=np.float64)
        self.len = len(self.df)- max_step_per_episode-1

    def reset(self):
        # global episode_reward
        self.steps = 0
        self.reward=0.0
        self.price_action = []
        self.start_point = random.randint(0, self.len-2)
        self.market = self.df.iloc[self.start_point]['close']
        state = self.get_state(self.start_point)
        # print(f'states: {state}')
        return state

    def step(self, action):
        global episode_reward, running_reward
        info = "Ramin Madani developed this Environment"
        call_margin = False
        # Get current state
        self.states = self.get_state(self.start_point)
        # print(self.states)
        self.market = self.states['close']
        if self.states['buy_profit'] + self.states['sell_profit'] < -100:
            call_margin = True

        # market movement
        self.start_point +=1

        # Execute action
        if call_margin:
            print(f'{RED}Call Margin{RESET}')
            self.done=True
            self.reward = self.states['buy_profit'] + self.states['sell_profit']
            next_state = self.states
        else:
            next_state = self.execute_action(action,self.market )

            if self.done:
                episode_reward += self.reward
                running_reward += episode_reward
            else:
                if action in [4,5]:
                    episode_reward += self.reward
                    # running_reward += self.reward
                elif action in [0]:
                    close = next_state['close']
                    buy_profit, sell_profit = self.calculate_profits(close)
                    self.reward +=  (buy_profit+sell_profit)

        return next_state, self.reward, self.done, info

    def get_state(self, nth_point):
        states = self.df.iloc[nth_point]
        buy_profit, sell_profit = self.calculate_profits(states['close'])
        states['buy_profit'] = buy_profit
        states['sell_profit'] = sell_profit
        return states

    def calculate_profits(self, close):
        buy_profit = sum(self.lot *(close - item[1]) for item in self.price_action if item[0] == 1)
        sell_profit = sum(self.lot * (item[1] - close) for item in self.price_action if item[0] == 2)
        return buy_profit, sell_profit

    def execute_action(self, action, before_price):
        self.market = self.get_state(self.start_point)['close']
        if action == 0:  #0 = nothing
            before_profit = self.calculate_profits(before_price)[0] + self.calculate_profits(before_price)[1]
            now_profit = self.calculate_profits(self.market)[0] + self.calculate_profits(self.market)[1]
            self.reward = now_profit-before_profit
            self.steps += 1

        elif action == 1:  #1 = buy
            self.price_action.append([1, before_price])
            self.reward = 0.5 # living penalty
            self.reward += round(self.calculate_profits(self.market)[0] + self.calculate_profits(self.market)[1] - self.calculate_profits(before_price)[0] - self.calculate_profits(before_price)[1], 2)
            self.steps += 1

        elif action == 2:  #2 = sell
            self.price_action.append([2, before_price])
            self.reward = 0.5 # living penalty
            self.reward += round(self.calculate_profits(self.market)[0] + self.calculate_profits(self.market)[1] - self.calculate_profits(before_price)[0] - self.calculate_profits(before_price)[1], 2)
            self.steps += 1

        elif action == 3:  #3 = close all
            self.reward = self.close_all_positions(before_price)

        elif action == 4:  #4 = close buy
            self.reward = self.close_positions(1,before_price)

        elif action == 5:   #5 = close sell
            self.reward = self.close_positions(2,before_price)

        next_state = self.get_state(self.start_point)
        self.market = next_state['close']
        return next_state

    def close_all_positions(self,befor_price):
        reward = sum(self.lot * (befor_price - item[1]) if item[0] == 1 else self.lot * (item[1] - befor_price) for item in self.price_action)
        self.price_action.clear()
        self.reward = 0.0
        self.done = True
        return reward

    def close_positions(self, position_type,befor_price):
        reward = 0.0
        remaining_positions = []
        for item in self.price_action:
            if item[0] == position_type:
                reward += self.lot * (befor_price - item[1]) if position_type == 1 else self.lot * (item[1] - befor_price)
            else:
                remaining_positions.append(item)
        self.price_action.clear()
        self.price_action = remaining_positions
        self.steps += 1
        if len(remaining_positions) ==0:
            self.done = True
            self.reward = 0.0
        return reward

    ''' 0 = nothing
    1 = buy
    2 = sell
    3 = close all
    4 = close buy
    5 = close sell'''

# Replay buffer

In [12]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0


    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)

# Agent

In [ ]:
import numpy as np
import random
import time
import tensorflow as tf
import keras
from keras import layers
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

random.seed(42)
# Hyperparameters
gamma = 0.9
clip_ratio = 0.2
actor_lr = 1.0
critic_lr = 1.0
batch_size = 512
num_epochs = 20


YELLOW = "\033[33m"
RESET = "\033[0m"

# PPO Agent
class PPOAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.epsilon = 1e-9
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        self.actor_optimizer = keras.optimizers.Adadelta(actor_lr)
        self.critic_optimizer = keras.optimizers.Adadelta(critic_lr)
        self.actor_loss = tf.keras.metrics.Mean('actor_loss', dtype=tf.float32)
        self.actor_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('actor_accuracy')
        self.critic_loss = tf.keras.metrics.Mean('critic_loss', dtype=tf.float32)
        self.critic_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('critic_accuracy')

    def build_actor(self):
        keras.regularizers.L2(0.001)
        state_input = layers.Input(shape=(self.state_dim,))
        dense1 = layers.Dense(1024,kernel_regularizer='L2', activation='linear')(state_input)
        dense2 = layers.Dense(1024,kernel_regularizer='L2', activation='tanh')(dense1)
        # dense3 = layers.Dense(512,kernel_regularizer='L1L2', activation='relu')(dense2)
        # dense4 = layers.Dense(256,kernel_regularizer='L1L2', activation='selu')(dense3)
        output = layers.Dense(self.action_dim, activation='softmax')(dense2)
        model = keras.Model(inputs=state_input, outputs=output)
        # dense1.trainable = False
        return model

    def build_critic(self):
        keras.regularizers.L2(0.001)
        state_input = layers.Input(shape=(self.state_dim,))
        dense1 = layers.Dense(1024,kernel_regularizer='L2', activation='linear')(state_input)
        dense2 = layers.Dense(1024,kernel_regularizer='L2', activation='tanh')(dense1)
        # dense3 = layers.Dense(512,kernel_regularizer='L1L2', activation='linear')(dense2)
        # dense4 = layers.Dense(256,kernel_regularizer='L1L2', activation='softplus')(dense3)
        output = layers.Dense(1)(dense2)
        model = keras.Model(inputs=state_input, outputs=output)
        # dense1.trainable = False
        return model

    def save_model(self, filepath, **kwargs):
        self.actor.save(filepath +"actor.keras")
        self.critic.save(filepath +"critic.keras")
        print(f"Model saved to {filepath}")

    def load_model(self, filepath, **kwargs):
        try:
            self.actor = keras.models.load_model(filepath +"actor.keras")
            self.critic = keras.models.load_model(filepath +"critic.keras")
            print(f"Model loaded from {filepath}")
        except Exception as e:
            print(f"An error occurred during LOAD Model: {e}")

    def get_action(self, state):
        state = np.expand_dims(state, axis=0)
        action_probs = self.actor(state)
        action = np.random.choice(self.action_dim, p=action_probs.numpy()[0])
        return action

    def normalizer(self, data):
        normalizer_layer = tf.keras.layers.Normalization(axis=-1)
        normalizer_layer.adapt(data)
        return normalizer_layer(data).numpy()


    def train(self, replay_buffer):
        if len(replay_buffer) < batch_size:
            return


        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        for epoch in range(num_epochs):
            with tf.GradientTape(persistent=True) as tape:
                # Critic loss
                target_q = reward + gamma * (1 - done) * self.critic(next_state)
                current_q = self.critic(state)
                critic_losses = tf.reduce_mean(tf.square(current_q - target_q))

                # Actor loss
                action_probs = self.actor(state)
                action_one_hot = tf.one_hot(action, self.action_dim)
                selected_action_probs = tf.reduce_sum(action_probs * action_one_hot, axis=1)
                old_action_probs = tf.stop_gradient(selected_action_probs) + self.epsilon
                ratio = selected_action_probs / old_action_probs
                surr1 = ratio * reward
                surr2 = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio) * reward
                actor_losses = -tf.reduce_mean(tf.minimum(surr1, surr2))

            # Update networks
            actor_grads = tape.gradient(actor_losses, self.actor.trainable_variables)
            critic_grads = tape.gradient(critic_losses, self.critic.trainable_variables)

            self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_weights))
            self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_weights))

            self.actor_loss(actor_losses)
            self.critic_loss(critic_losses)
            self.actor_accuracy(surr1,surr2)
            self.critic_accuracy(target_q,current_q)
        print(f'actor_loss:{actor_losses:.5f}  critic_loss:{critic_losses:.5f}')
###############################################################################################



# Play The Game

In [ ]:
import os
import numpy as np
import pandas as pd
from environment import TrainForex, RealForex, get_data_bollinger, get_data_ichimoku
from Agent import PPOAgent, PPOAgent_Tuner, ReplayBuffer
import tensorflow as tf


CYAN = "\033[36m"
RESET = "\033[0m"

buffer_capacity = 1000000
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M15  # 15-minute timeframe
path = f'PPO/v 1.0/Saved Models/model_{symbol}_{timeframe}_'
num_episodes = 1000
max_step_per_episode = 50  # Example value, adjust as needed
episode_reward = 0.0
running_reward = 0.0
best_reward = 0.0
actions_row = []

# Set random seeds for reproducibility
np.random.seed(42)

# Set option to display all columns
pd.set_option('display.max_columns', None)
# This approach ensures that all columns of the DataFrame are visible when printed in the terminal.
pd.reset_option('display.max_columns')

# Get data
df = get_data_ichimoku(symbol, timeframe,number_of_candles=40000) #,max_period=25)

# Initialize train environment
env = TrainForex(max_step_per_episode=max_step_per_episode, df = df)  # Example environment, replace with actual
num_actions = env.action_space.n
print(num_actions)


state_dim = env.states.shape[0]
action_dim = env.action_space.n

agent = PPOAgent(state_dim, action_dim)
if os.path.exists('PPO/v 1.0/Saved Models/'):
    agent.load_model(path)
replay_buffer = ReplayBuffer(buffer_capacity)

actor_log_dir = 'logs/gradient_tape/' +  '/actor'
critic_log_dir = 'logs/gradient_tape/' +  '/critic'
actor_summary_writer = tf.summary.create_file_writer(actor_log_dir)
critic_summary_writer = tf.summary.create_file_writer(critic_log_dir)

# Training loop

# def train_agent(agent, env, replay_buffer, num_episodes, max_step_per_episode, running_reward, actions_row):
for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    env.reward = 0.0
    env.done = False
    done = False
    # for action in actions_row:
    while True:
        if env.steps == 0:
            action = np.random.randint(1, 3)
        elif env.steps >= max_step_per_episode:
            action = 3
        else:
            action = agent.get_action(state)

        actions_row.append(action)
        next_state, reward, done, _ = env.step(action)
        if reward>0:
            reward /=1
        replay_buffer.push(state, action, reward, next_state, done)
        state = next_state
        if done:
            episode_reward = env.reward
            running_reward += episode_reward
            print(f"{CYAN}Episode: {episode}, Reward: {episode_reward:.2f}  Running Reward: {running_reward:.2f}  Actions: {actions_row} {len(actions_row)} {RESET}")
            agent.train(replay_buffer)
            with actor_summary_writer.as_default():
                tf.summary.scalar('loss', agent.actor_loss.result(), step=episode)
                tf.summary.scalar('accuracy', agent.actor_accuracy.result(), step=episode)
            with critic_summary_writer.as_default():
                tf.summary.scalar('loss', agent.critic_loss.result(), step=episode)
                tf.summary.scalar('accuracy', agent.critic_accuracy.result(), step=episode)

            actions_row.clear()
            break
    # Reset metrics every epoch
    agent.actor_loss.reset_state()
    agent.critic_loss.reset_state()
    agent.actor_accuracy.reset_state()
    agent.critic_accuracy.reset_state()

agent.save_model(path)